# Extract Features

1. Read csv files and load as dfs
2. Combine dfs
3. Get semantic cosine similarity

In [1]:
import os, sys

import pandas as pd

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from log_files import LogData
from data_processing import DataProcessing
from feature_extraction import SpacyFeatureExtraction

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Predictions

- Use the structure from `1-generate_predictions-all_domains.ipynb`

In [3]:
log_file_path = "data/prediction_logs"
predictions = True
predictions_df = log_files.read_data(notebook_dir, log_file_path, predictions)
predictions_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs/batch_1-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs/batch_1-prediction/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs/batch_2-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs/batch_2-prediction/batch_2-from_df.csv


,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,1,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


## Observations

In [4]:
log_file_path = "data/observation_logs"
predictions = False
observations_df = log_files.read_data(notebook_dir, log_file_path, predictions)
observations_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs/batch_1-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs/batch_1-observation/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs/batch_2-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs/batch_2-observation/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/m

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"A young investor predicts on 2025-03-15, the S&P 500 index may rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 2027-01-01 to 2027-12-31, Wells Fargo envisions that the interest rates at the Federal Reserve have some probability to remain stable.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan observed that the net profit at Microsoft had risen in September 2023.,0,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


## Both

- Create a knowledge graph
    - Nodes: words
    - Edges: connection to other words (same/diff sentence)
- Look at code from Graphbreeding project on 2019 Mac

In [5]:
df = DataProcessing.concat_dfs([predictions_df, observations_df])
df.head(3)

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3


In [6]:
predictions = DataProcessing.df_to_list(predictions_df, "Base Sentence")
observations = DataProcessing.df_to_list(observations_df, "Base Sentence")

In [7]:
predictions

['JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.',
 'On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.',
 'Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.',
 'According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.',
 'In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.',
 'The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.',
 'JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.',
 'On August 25, 2024, to September 25, 2025, Citigroup speculates the net profit at Johnson & Johnson will likely increase.',
 'Bank of America predicts on 2024-08-21, the operating income at Visa may rise.',
 'According to Goldman Sachs, the research and development expenses at Alphabet would fall in 2029 Q2.',
 'In 

In [ ]:
disable_components = ["lemmatizer"]
spacy_fe = SpacyFeatureExtraction(predictions_df, "Base Sentence")
all_pos_tags, tags, all_ner_tags, entities = spacy_fe.extract_features(disable_components)

Spacy Doc (0):  JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.
Spacy Doc (1):  On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.
Spacy Doc (2):  Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.
Spacy Doc (3):  According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.


# Mapping variables : words in sentence(s)

- Sentenc/Spacy Doc (0): JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.
    1. $ p_s $: JPMorgan Chase
    2. $ p_t $: Amazon
    3. $ p_d $: Q3 of 2027
    4. $ p_a $: net profit

- Sentence/Spacy Doc (1):  On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.
    1. $ p_s $: JPMorgan Chase
    2. $ p_t $: Amazon
    3. $ p_d $: Q3 of 2027
    4. $ p_a $: net profit


In [13]:
pos_df = DataProcessing.convert_tags_entities_to_dataframe(all_pos_tags, tags)
pos_df.head(7)

,NOUN_4,DET_5,PUNCT_3,NOUN_10,SCONJ_1,ADP_5,PUNCT_2,PROPN_1,CCONJ_1,PUNCT_1,NOUN_3,PUNCT_4,PRON_1,ADJ_4,SYM_2,VERB_2,CCONJ_2,NUM_4,NOUN_5,NUM_2,NOUN_8,DET_4,PROPN_6,NOUN_1,NUM_3,PROPN_5,NOUN_11,ADP_4,NOUN_6,NOUN_7,VERB_4,NUM_1,ADP_3,ADV_1,DET_3,PUNCT_5,NOUN_9,ADJ_1,VERB_1,ADP_1,PART_1,ADP_6,PROPN_2,ADJ_2,ADJ_3,VERB_3,AUX_1,DET_2,DET_1,PROPN_7,SYM_1,PROPN_3,ADP_2,NOUN_2,PROPN_4,PRON_2
0,NaN,NaN,NaN,NaN,that,NaN,NaN,JPMorgan,NaN,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,profit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2027,of,potentially,NaN,NaN,NaN,net,forecasts,at,NaN,NaN,Chase,NaN,NaN,NaN,NaN,NaN,the,NaN,NaN,Amazon,in,decrease,Q3,NaN
1,NaN,NaN,.,NaN,NaN,NaN,",",August,NaN,",",NaN,NaN,NaN,NaN,NaN,increase,NaN,NaN,NaN,2024,NaN,NaN,NaN,revenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,at,likely,NaN,NaN,NaN,NaN,speculates,On,NaN,NaN,Bank,NaN,NaN,NaN,will,NaN,the,NaN,NaN,America,of,NaN,Microsoft,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,.,Citigroup,NaN,",",NaN,NaN,NaN,NaN,-,rise,NaN,NaN,NaN,08,NaN,NaN,NaN,operating,21,NaN,NaN,NaN,NaN,NaN,NaN,2024,NaN,NaN,NaN,NaN,NaN,NaN,predicts,on,NaN,NaN,Alphabet,NaN,NaN,NaN,may,NaN,the,NaN,-,NaN,at,income,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,.,Goldman,and,",",expenses,NaN,NaN,NaN,NaN,fall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,research,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,in,NaN,NaN,NaN,NaN,NaN,According,to,NaN,NaN,Sachs,NaN,NaN,NaN,would,NaN,the,NaN,NaN,Facebook,at,development,NaN,NaN
4,NaN,NaN,NaN,NaN,that,NaN,.,August,&,",",NaN,NaN,NaN,NaN,NaN,has,NaN,NaN,NaN,2024,NaN,NaN,NaN,profit,NaN,Johnson,NaN,NaN,NaN,NaN,NaN,21,NaN,NaN,NaN,NaN,NaN,gross,envisions,In,to,NaN,Morgan,stable,NaN,remain,NaN,some,the,NaN,NaN,Stanley,at,probability,Johnson,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,.,Visa,NaN,",",NaN,NaN,NaN,NaN,NaN,according,NaN,NaN,NaN,NaN,NaN,NaN,NaN,stock,NaN,NaN,NaN,to,NaN,NaN,NaN,2026,of,NaN,NaN,NaN,NaN,same,stay,at,NaN,NaN,Q2,NaN,NaN,NaN,should,NaN,The,NaN,NaN,Wells,in,price,Fargo,NaN
6,NaN,NaN,NaN,NaN,that,NaN,NaN,JPMorgan,NaN,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,revenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2027,of,potentially,NaN,NaN,NaN,NaN,forecasts,at,NaN,NaN,Microsoft,NaN,NaN,NaN,NaN,NaN,the,NaN,NaN,Q3,in,decrease,NaN,NaN


In [14]:
ner_df = DataProcessing.convert_tags_entities_to_dataframe(all_ner_tags, entities)
ner_df.head(7)

,DATE_2,PRODUCT_1,GPE_2,TIME_1,ORG_1,ORG_2,GPE_1,PERSON_1,FAC_1,MONEY_1,LOC_1,DATE_1,CARDINAL_1,NORP_1,PERSON_2,EVENT_1,ORG_3
0,NaN,NaN,NaN,NaN,JPMorgan Chase,Amazon,Q3,NaN,NaN,NaN,NaN,2027,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Bank of America,Microsoft,NaN,NaN,NaN,NaN,NaN,"August 21, 2024",NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,Citigroup,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-21,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,Goldman Sachs,Facebook,NaN,NaN,NaN,NaN,NaN,2025,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,Morgan Stanley,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,21 August 2024,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,Visa,Q2,NaN,NaN,NaN,NaN,NaN,2026,NaN,NaN,NaN,NaN,Wells Fargo
6,NaN,NaN,NaN,NaN,JPMorgan,Microsoft,Q3,NaN,NaN,NaN,NaN,2027,NaN,NaN,NaN,NaN,NaN
